In [23]:
from models import *
from utils import  *
import torch
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
from tqdm import tqdm

### Step 1: Obtain the student and teacher models which allow feature map extraction

In [24]:
VGG16_teacher = vgg16_bn_f(False,100)
VGG16_teacher.load_state_dict(torch.load('teacher_model_vgg16_bn.pth'))

<All keys matched successfully>

In [25]:
VGG11_student = vgg11_bn_f(False,100)

Distill using FitNets

In [26]:
fitnet = create_model(VGG16_teacher, VGG11_student, 1,0.75)

In [27]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
fitnet.teacher = fitnet.teacher.to(device)
fitnet.student = fitnet.student.to(device)
fitnet.regressor = fitnet.regressor.to(device)
EPOCHS = 30
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(fitnet.get_learnable_params(), lr=0.002)
tr_metric = {"Accuracy":[],"Loss":[]}
ts_metric = {"Accuracy":[],"Loss":[]}

for epoch in range(EPOCHS):
    tr_loss, tr_acc = train_hints(fitnet,train_loader,optimizer,device)
    tr_metric["Accuracy"].append(tr_acc)
    tr_metric["Loss"].append(tr_loss)

    print("Epoch: {} Train Loss: {:.2f} Train Accuracy: {:.2f}".format(epoch+1, tr_loss, tr_acc))
ts_loss, ts_acc = eval_hints(fitnet,test_loader,device)
ts_metric["Accuracy"].append(ts_acc)
ts_metric["Loss"].append(ts_loss)
print("Epoch: {} Train Loss: {:.2f} Train Accuracy: {:.2f} Test Loss: {:.2f} Test Accuracy: {:.2f}".format(epoch+1, tr_loss, tr_acc, ts_loss, ts_acc))


Epoch: 1 Train Loss: 1.10 Train Accuracy: 0.03
Epoch: 2 Train Loss: 1.05 Train Accuracy: 0.04
Epoch: 3 Train Loss: 1.03 Train Accuracy: 0.05
Epoch: 4 Train Loss: 0.99 Train Accuracy: 0.06
Epoch: 5 Train Loss: 0.96 Train Accuracy: 0.08
Epoch: 6 Train Loss: 0.93 Train Accuracy: 0.09
Epoch: 7 Train Loss: 0.90 Train Accuracy: 0.10
Epoch: 8 Train Loss: 0.88 Train Accuracy: 0.12
Epoch: 9 Train Loss: 0.86 Train Accuracy: 0.13
Epoch: 10 Train Loss: 0.83 Train Accuracy: 0.15
Epoch: 11 Train Loss: 0.81 Train Accuracy: 0.16
Epoch: 12 Train Loss: 0.79 Train Accuracy: 0.17
Epoch: 13 Train Loss: 0.78 Train Accuracy: 0.19
Epoch: 14 Train Loss: 0.76 Train Accuracy: 0.20
Epoch: 15 Train Loss: 0.74 Train Accuracy: 0.22
Epoch: 16 Train Loss: 0.72 Train Accuracy: 0.23
Epoch: 17 Train Loss: 0.71 Train Accuracy: 0.24
Epoch: 18 Train Loss: 0.70 Train Accuracy: 0.25
Epoch: 19 Train Loss: 0.68 Train Accuracy: 0.27
Epoch: 20 Train Loss: 0.67 Train Accuracy: 0.28
Epoch: 21 Train Loss: 0.66 Train Accuracy: 0.29
E

In [28]:
print("Final Train Accuracy:",tr_metric["Accuracy"][-1])
print("Final Test Accuracy:",ts_metric["Accuracy"][-1])
print("Final Train Loss:",tr_metric["Loss"][-1])
print("Final Test Loss:",ts_metric["Loss"][-1])

Final Train Accuracy: 0.39348
Final Test Accuracy: 0.39229998
Final Train Loss: 0.5512283030906906
Final Test Loss: 2.2636840647193277


In [29]:
import pandas as pd
pd.DataFrame(tr_metric).to_csv("vgg11_cifar100_hint_tr.csv") 
pd.DataFrame(ts_metric).to_csv("vgg11_cifar100_hint_ts.csv")

In [30]:
torch.save(fitnet.student.state_dict(),"vgg11_cifar100_hint_student.pth")
torch.save(fitnet.regressor.state_dict(),"vgg11_cifar100_hint_regressor.pth")

In [45]:
VGG16_teacher = vgg16_bn_f(False,100)
VGG16_teacher.load_state_dict(torch.load('teacher_model_vgg16_bn.pth'))

<All keys matched successfully>

In [46]:
VGG11_student = vgg11_bn_f(False,100)

In [47]:
fitnet = create_model(VGG16_teacher, VGG11_student, 0,0.75)

In [48]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
fitnet.teacher = fitnet.teacher.to(device)
fitnet.student = fitnet.student.to(device)
fitnet.regressor = fitnet.regressor.to(device)
EPOCHS = 30
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(fitnet.get_learnable_params(), lr=0.002)
tr_metric = {"Accuracy":[],"Loss":[]}
ts_metric = {"Accuracy":[],"Loss":[]}

for epoch in range(EPOCHS):
    tr_loss, tr_acc = train_hints(fitnet,train_loader,optimizer,device)
    tr_metric["Accuracy"].append(tr_acc)
    tr_metric["Loss"].append(tr_loss)

    print("Epoch: {} Train Loss: {:.2f} Train Accuracy: {:.2f}".format(epoch+1, tr_loss, tr_acc))
ts_loss, ts_acc = eval_hints(fitnet,test_loader,device)
ts_metric["Accuracy"].append(ts_acc)
ts_metric["Loss"].append(ts_loss)
print("Epoch: {} Train Loss: {:.2f} Train Accuracy: {:.2f} Test Loss: {:.2f} Test Accuracy: {:.2f}".format(epoch+1, tr_loss, tr_acc, ts_loss, ts_acc))


Epoch: 1 Train Loss: 1.09 Train Accuracy: 0.03
Epoch: 2 Train Loss: 1.03 Train Accuracy: 0.05
Epoch: 3 Train Loss: 1.00 Train Accuracy: 0.06
Epoch: 4 Train Loss: 0.97 Train Accuracy: 0.08
Epoch: 5 Train Loss: 0.94 Train Accuracy: 0.09
Epoch: 6 Train Loss: 0.91 Train Accuracy: 0.11
Epoch: 7 Train Loss: 0.89 Train Accuracy: 0.12
Epoch: 8 Train Loss: 0.87 Train Accuracy: 0.13
Epoch: 9 Train Loss: 0.85 Train Accuracy: 0.14
Epoch: 10 Train Loss: 0.83 Train Accuracy: 0.16
Epoch: 11 Train Loss: 0.82 Train Accuracy: 0.17
Epoch: 12 Train Loss: 0.80 Train Accuracy: 0.18
Epoch: 13 Train Loss: 0.78 Train Accuracy: 0.19
Epoch: 14 Train Loss: 0.76 Train Accuracy: 0.21
Epoch: 15 Train Loss: 0.75 Train Accuracy: 0.22
Epoch: 16 Train Loss: 0.73 Train Accuracy: 0.23
Epoch: 17 Train Loss: 0.71 Train Accuracy: 0.24
Epoch: 18 Train Loss: 0.70 Train Accuracy: 0.26
Epoch: 19 Train Loss: 0.68 Train Accuracy: 0.27
Epoch: 20 Train Loss: 0.67 Train Accuracy: 0.28
Epoch: 21 Train Loss: 0.65 Train Accuracy: 0.30
E

In [49]:
print("Final Train Accuracy:",tr_metric["Accuracy"][-1])
print("Final Test Accuracy:",ts_metric["Accuracy"][-1])
print("Final Train Loss:",tr_metric["Loss"][-1])
print("Final Test Loss:",ts_metric["Loss"][-1])

Final Train Accuracy: 0.39606
Final Test Accuracy: 0.4233
Final Train Loss: 0.5538564060273987
Final Test Loss: 2.173304708140671


In [50]:
import pandas as pd
pd.DataFrame(tr_metric).to_csv("vgg11_cifar100_hint_tr.csv") 
pd.DataFrame(ts_metric).to_csv("vgg11_cifar100_hint_ts.csv")

In [51]:
torch.save(fitnet.student.state_dict(),"vgg11_cifar100_hint_student.pth")
torch.save(fitnet.regressor.state_dict(),"vgg11_cifar100_hint_regressor.pth")

In [38]:
VGG16_teacher = vgg16_bn_f(False,100)
VGG16_teacher.load_state_dict(torch.load('teacher_model_vgg16_bn.pth'))

<All keys matched successfully>

In [39]:
VGG11_student = vgg11_bn_f(False,100)

In [40]:
fitnet = create_model(VGG16_teacher, VGG11_student, 2,0.75)

In [41]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
fitnet.teacher = fitnet.teacher.to(device)
fitnet.student = fitnet.student.to(device)
fitnet.regressor = fitnet.regressor.to(device)
EPOCHS = 30
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(fitnet.get_learnable_params(), lr=0.002)
tr_metric = {"Accuracy":[],"Loss":[]}
ts_metric = {"Accuracy":[],"Loss":[]}

for epoch in range(EPOCHS):
    tr_loss, tr_acc = train_hints(fitnet,train_loader,optimizer,device)
    tr_metric["Accuracy"].append(tr_acc)
    tr_metric["Loss"].append(tr_loss)

    print("Epoch: {} Train Loss: {:.2f} Train Accuracy: {:.2f}".format(epoch+1, tr_loss, tr_acc))
ts_loss, ts_acc = eval_hints(fitnet,test_loader,device)
ts_metric["Accuracy"].append(ts_acc)
ts_metric["Loss"].append(ts_loss)
print("Epoch: {} Train Loss: {:.2f} Train Accuracy: {:.2f} Test Loss: {:.2f} Test Accuracy: {:.2f}".format(epoch+1, tr_loss, tr_acc, ts_loss, ts_acc))


Epoch: 1 Train Loss: 1.09 Train Accuracy: 0.03
Epoch: 2 Train Loss: 1.04 Train Accuracy: 0.04
Epoch: 3 Train Loss: 1.01 Train Accuracy: 0.05
Epoch: 4 Train Loss: 0.99 Train Accuracy: 0.06
Epoch: 5 Train Loss: 0.97 Train Accuracy: 0.07
Epoch: 6 Train Loss: 0.94 Train Accuracy: 0.08
Epoch: 7 Train Loss: 0.92 Train Accuracy: 0.09
Epoch: 8 Train Loss: 0.90 Train Accuracy: 0.10
Epoch: 9 Train Loss: 0.88 Train Accuracy: 0.12
Epoch: 10 Train Loss: 0.86 Train Accuracy: 0.13
Epoch: 11 Train Loss: 0.84 Train Accuracy: 0.14
Epoch: 12 Train Loss: 0.82 Train Accuracy: 0.15
Epoch: 13 Train Loss: 0.81 Train Accuracy: 0.17
Epoch: 14 Train Loss: 0.79 Train Accuracy: 0.18
Epoch: 15 Train Loss: 0.77 Train Accuracy: 0.19
Epoch: 16 Train Loss: 0.75 Train Accuracy: 0.21
Epoch: 17 Train Loss: 0.73 Train Accuracy: 0.22
Epoch: 18 Train Loss: 0.72 Train Accuracy: 0.24
Epoch: 19 Train Loss: 0.70 Train Accuracy: 0.25
Epoch: 20 Train Loss: 0.69 Train Accuracy: 0.26
Epoch: 21 Train Loss: 0.67 Train Accuracy: 0.27
E

In [42]:
print("Final Train Accuracy:",tr_metric["Accuracy"][-1])
print("Final Test Accuracy:",ts_metric["Accuracy"][-1])
print("Final Train Loss:",tr_metric["Loss"][-1])
print("Final Test Loss:",ts_metric["Loss"][-1])

Final Train Accuracy: 0.3732
Final Test Accuracy: 0.3642
Final Train Loss: 0.5679276254780762
Final Test Loss: 2.3632015445429806


In [43]:
import pandas as pd
pd.DataFrame(tr_metric).to_csv("vgg11_cifar100_hint_tr.csv") 
pd.DataFrame(ts_metric).to_csv("vgg11_cifar100_hint_ts.csv")

In [44]:
torch.save(fitnet.student.state_dict(),"vgg11_cifar100_hint_student.pth")
torch.save(fitnet.regressor.state_dict(),"vgg11_cifar100_hint_regressor.pth")

In [52]:
VGG16_teacher = vgg16_bn_f(False,100)
VGG16_teacher.load_state_dict(torch.load('teacher_model_vgg16_bn.pth'))

<All keys matched successfully>

In [53]:
VGG11_student = vgg11_bn_f(False,100)

In [54]:
fitnet = create_model(VGG16_teacher, VGG11_student, 3,0.75)

In [55]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
fitnet.teacher = fitnet.teacher.to(device)
fitnet.student = fitnet.student.to(device)
fitnet.regressor = fitnet.regressor.to(device)
EPOCHS = 30
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(fitnet.get_learnable_params(), lr=0.002)
tr_metric = {"Accuracy":[],"Loss":[]}
ts_metric = {"Accuracy":[],"Loss":[]}

for epoch in range(EPOCHS):
    tr_loss, tr_acc = train_hints(fitnet,train_loader,optimizer,device)
    tr_metric["Accuracy"].append(tr_acc)
    tr_metric["Loss"].append(tr_loss)

    print("Epoch: {} Train Loss: {:.2f} Train Accuracy: {:.2f}".format(epoch+1, tr_loss, tr_acc))
ts_loss, ts_acc = eval_hints(fitnet,test_loader,device)
ts_metric["Accuracy"].append(ts_acc)
ts_metric["Loss"].append(ts_loss)
print("Epoch: {} Train Loss: {:.2f} Train Accuracy: {:.2f} Test Loss: {:.2f} Test Accuracy: {:.2f}".format(epoch+1, tr_loss, tr_acc, ts_loss, ts_acc))


Epoch: 1 Train Loss: 1.08 Train Accuracy: 0.03
Epoch: 2 Train Loss: 1.03 Train Accuracy: 0.05
Epoch: 3 Train Loss: 1.00 Train Accuracy: 0.06
Epoch: 4 Train Loss: 0.97 Train Accuracy: 0.07
Epoch: 5 Train Loss: 0.94 Train Accuracy: 0.08
Epoch: 6 Train Loss: 0.91 Train Accuracy: 0.10
Epoch: 7 Train Loss: 0.89 Train Accuracy: 0.11
Epoch: 8 Train Loss: 0.86 Train Accuracy: 0.13
Epoch: 9 Train Loss: 0.84 Train Accuracy: 0.15
Epoch: 10 Train Loss: 0.81 Train Accuracy: 0.17
Epoch: 11 Train Loss: 0.79 Train Accuracy: 0.18
Epoch: 12 Train Loss: 0.77 Train Accuracy: 0.20
Epoch: 13 Train Loss: 0.75 Train Accuracy: 0.22
Epoch: 14 Train Loss: 0.73 Train Accuracy: 0.23
Epoch: 15 Train Loss: 0.71 Train Accuracy: 0.25
Epoch: 16 Train Loss: 0.70 Train Accuracy: 0.26
Epoch: 17 Train Loss: 0.68 Train Accuracy: 0.28
Epoch: 18 Train Loss: 0.67 Train Accuracy: 0.29
Epoch: 19 Train Loss: 0.65 Train Accuracy: 0.30
Epoch: 20 Train Loss: 0.64 Train Accuracy: 0.31
Epoch: 21 Train Loss: 0.62 Train Accuracy: 0.33
E

In [56]:
print("Final Train Accuracy:",tr_metric["Accuracy"][-1])
print("Final Test Accuracy:",ts_metric["Accuracy"][-1])
print("Final Train Loss:",tr_metric["Loss"][-1])
print("Final Test Loss:",ts_metric["Loss"][-1])

Final Train Accuracy: 0.43023998
Final Test Accuracy: 0.4353
Final Train Loss: 0.5205932102544838
Final Test Loss: 2.1014140039492566


In [59]:
import pandas as pd
pd.DataFrame(tr_metric).to_csv("vgg11_cifar100_hint_tr.csv") 
pd.DataFrame(ts_metric).to_csv("vgg11_cifar100_hint_ts.csv")

In [60]:
torch.save(fitnet.student.state_dict(),"vgg11_cifar100_hint_student.pth")
torch.save(fitnet.regressor.state_dict(),"vgg11_cifar100_hint_regressor.pth")